In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

!rm -rf ./logs/cifar10/*

In [3]:
train_ds, test_ds = tfds.load('cifar10', split=['train','test'], as_supervised=True, batch_size = -1)

x_train, y_train = tfds.as_numpy(train_ds)
x_test, y_test = tfds.as_numpy(test_ds)

n = x_train.shape[0]
n_dims = tf.math.cumprod(x_train.shape[1:])[-1].numpy()

# Preprocess the data by flattening & scaling it
x_train = x_train.reshape(-1, 32*32*3).astype("float32") / 255
x_test = x_test.reshape(-1, 32*32*3).astype("float32") / 255

# Categorical (one hot) encoding of the labels
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

_, n_classes = y_train.shape

print((n, n_dims, n_classes))

(50000, 3072, 10)


In [6]:
epochs = 20
batch_size = 64

In [7]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model_svm = keras.Sequential(
    [
        keras.Input(shape=(n_dims,)),
        RandomFourierFeatures(
            output_dim=8192, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=n_classes),
    ]
)
model_svm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

modeldir = "./logs/cifar10/linear-8192-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_svm.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[
        keras.callbacks.TensorBoard(
            log_dir=modeldir+"/log",
            histogram_freq=1,
        )
    ],
)
model_svm.save(modeldir + "/model")

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2930 - acc: 0.3924 - val_loss: 0.2764 - val_acc: 0.4234
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2413 - acc: 0.5092 - val_loss: 0.2957 - val_acc: 0.4271
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2257 - acc: 0.5569 - val_loss: 0.2740 - val_acc: 0.4677
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2133 - acc: 0.5901 - val_loss: 0.2877 - val_acc: 0.4603
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2049 - acc: 0.6100 - val_loss: 0.3096 - val_acc: 0.4426
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2017 - acc: 0.6273 - val_loss: 0.3104 - val_acc: 0.4471
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1941 - acc: 0.6436 - val_loss: 0.3172 - val_acc: 0.4455
Epoch 8/10
782/782 [==============================] - 5s 6ms/step - loss: 0.

INFO:tensorflow:Assets written to: ./logs/cifar10/linear-8192-20230306-105525/model\assets


In [ ]:
def gen_samples(X, psi, t=1000):
    n, _ = X.shape
    return [
        tf.gather_nd(
            indices=tf.expand_dims(tf.random.shuffle(tf.range(n))[:psi], 1), params=X
        ).numpy()
        for _ in range(t)
    ]

In [ ]:
def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
        super(IsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p
        self.soft = soft

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
                "soft": self.soft,
            }
        )
        return config

In [ ]:
def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], p=p, soft=soft, name="ann_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=10, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model

In [ ]:
t_samples = gen_samples(x_train, psi=16, t=500)


In [ ]:
model_hard_16_500 = build_model(t_samples, soft=False)
modeldir = "./logs/cifar10/hard-16x500-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_16_500.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_16_500.save(modeldir + "/model")

In [ ]:
model_soft_16_500 = build_model(t_samples, soft=True)
modeldir = "./logs/cifar10/soft-16x500-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_16_500.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_16_500.save(modeldir + "/model")

In [ ]:
t_samples = gen_samples(x_train, psi=160, t=50)

In [ ]:
model_hard_160_50 = build_model(t_samples, soft=False)
modeldir = "./logs/cifar10/hard-160x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_160_50.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_160_50.save(modeldir + "/model")

In [ ]:
model_soft_160_50 = build_model(t_samples, soft=True)
modeldir = "./logs/cifar10/soft-160x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_160_50.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_160_50.save(modeldir + "/model")

In [ ]:
%tensorboard --logdir ./logs/fit